# All Python packages

In [ ]:
!git clone https://github.com/Jacob12138xieyuan/EEG-Based-Emotion-Recognition-on-DEAP

Cloning into 'EEG-Based-Emotion-Recognition-on-DEAP'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 57 (delta 5), reused 47 (delta 1), pack-reused 0
Unpacking objects: 100% (57/57), done.


In [ ]:
!python /content/EEG-Based-Emotion-Recognition-on-DEAP/eeg_entropy.py

In [ ]:
!pip install eeg_entropy

ERROR: Could not find a version that satisfies the requirement eeg_entropy (from versions: none)
ERROR: No matching distribution found for eeg_entropy


In [ ]:
!pip3 install numpy
!pip3 install sklearn
!pip3 install scipy
!pip3 install matplotlib
!pip3 install tensorflow
!pip3 install keras
!pip3 install mne

In [ ]:
from IPython.utils import io
import numpy as np
import collections

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle

import scipy.io
from scipy import signal, integrate
import matplotlib.pyplot as plt

import keras
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Dropout

import mne
import eeg_entropy
import math

n_second = 60
n_segment = 2*n_second-1
n_points = n_second*128

ModuleNotFoundError: ignored

# Load np data and balance high and low label

In [ ]:
def load_np_data(dimension):
    if dimension == 'valence':
        all_labels, all_data = np.load('../Data/processed_DEAP/valence/' + 'all_valence_labels.npy', allow_pickle=True), np.load('../Data/processed_DEAP/valence/' + 'all_valence_data.npy', allow_pickle=True)
        print("Total valence: ", all_labels.shape, all_data.shape)
        #print("High and low valence: ", collections.Counter(all_labels))# 587 high valence, 472 low valence
    elif dimension == 'arousal':
        all_labels, all_data = np.load('../Data/processed_DEAP/arousal/' + 'all_arousal_labels.npy', allow_pickle=True), np.load('../Data/processed_DEAP/arousal/' + 'all_arousal_data.npy', allow_pickle=True)
        print("Total arousal: ", all_labels.shape, all_data.shape)
        #print("High and low arousal: ", collections.Counter(all_labels))# 620 high arousal, 462 low arousal
    return all_labels, all_data

In [ ]:
# all_labels, all_data = load_np_data(dimension="valence")
all_labels, all_data = load_np_data(dimension="arousal")

In [ ]:
# after standardised
print(np.amax(all_data)) # max value
print(np.amin(all_data)) # min value

# print(np.amax(all_valence_data[0])) # max value
# print(np.amin(all_valence_data[0])) # min value

# Feature extraction method (Power)

In [ ]:
def trial_psd_extraction_integration(data): # data shape (12, 8064)
    info = mne.create_info(ch_names= ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32'], sfreq=128);
    raw = mne.io.RawArray(data, info, first_samp=0, copy='auto', verbose=None);
    psd_origin, f = mne.time_frequency.psd_welch(raw, fmin=0, fmax=60, n_fft=128, n_overlap=64, n_per_seg=128, picks='all', window='hann', average=None, verbose=None)# average='mean' or None
    # print(psd_origin.shape, f.shape) # (12, 61, 125) (61,) 61 frequency
    psd = np.moveaxis(psd_origin, -1, 0) # (125, 12, 61)
    # calculate frequency band power using integration
    band_power = [] # band power for all segments
    for segment in psd:
        segment_band_power = [] # band power for all channels in one segment
        for psd_channel in segment:
            y_int = integrate.cumtrapz(psd_channel, f, initial=0) # integrate to calculate band power
            one_band_power = np.array([y_int[7]-y_int[4],y_int[13]-y_int[8],y_int[30]-y_int[14],y_int[51]-y_int[31]])
            segment_band_power.append(one_band_power)
        band_power.append(segment_band_power)
    band_power = np.array(band_power) # (125, 12, 4)
    band_power = np.moveaxis(band_power, -1, 1) # (125, 4, 12)
    band_power = band_power.reshape((n_segment, 32*4)) # flatten feature (125, 48)
    band_power = 10*band_power
    return band_power

# 10-fold cross-validation

In [ ]:
all_data, all_labels = shuffle(all_data, all_labels, random_state=0)
n = len(all_labels) # 1059
print(n)
fold_n = math.floor(n/10) # 105
print(fold_n)
all_data, all_labels = all_data[:10*fold_n], all_labels[:10*fold_n] # (1050, 32, 8064)
print(all_data.shape)

In [ ]:
def process(test_fold_number):
    # train has 9 folds, test has 1 fold
    train_data = np.concatenate((all_data[:test_fold_number*fold_n], all_data[fold_n+test_fold_number*fold_n:]), axis=0)
    train_labels = np.concatenate((all_labels[:test_fold_number*fold_n], all_labels[fold_n+test_fold_number*fold_n:]), axis=0)
    test_data = all_data[test_fold_number*fold_n : fold_n+test_fold_number*fold_n]
    test_labels = all_labels[test_fold_number*fold_n : fold_n+test_fold_number*fold_n]
    print(train_data.shape,test_data.shape) # (945, 32, 8064) (105, 32, 8064)

    # -------- Feature extraction from 32 original signal --------
    train_band_power = [] # band power feature sequence for train trials
    for data in train_data: # for every train trial
        with io.capture_output() as captured:
            trial_band_power = trial_psd_extraction_integration(data) # data shape (32, 8064)
        train_band_power.append(trial_band_power)
    train_band_power = np.array(train_band_power)

    test_band_power = [] # band power feature sequence for test trials
    for data in test_data: # for every test trial
        with io.capture_output() as captured:
            trial_band_power = trial_psd_extraction_integration(data) # data shape (32, 8064)
        test_band_power.append(trial_band_power)
    test_band_power = np.array(test_band_power)
    print("All features of training data shape: ", train_band_power.shape) # shape (849, 125, 128)
    print("All features of test data shape: ", test_band_power.shape) # shape (95, 125, 128)

    # -------- Create new LSTM model --------
    x=Input(shape=(n_segment,4*32)) # flatten (12,4) to 48
    x1=LSTM(n_segment)(x)
    x2=Dense(n_segment)(x1)
    x3=Dense(12)(x2)
    output=Dense(1, activation="sigmoid")(x2)
    model=Model(x, output)

    # -------- Compile and train LSTM --------
    model.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])
    history = model.fit(train_band_power, train_labels, epochs=30, batch_size=8, validation_data=(test_band_power, test_labels))
    print("Hightest accuracy: " + str(max(history.history['val_accuracy'])))
    model.save("../Results/LSTM_model/LSTM_model_test_fold_" + str(test_fold_number))


In [ ]:
for i in range(10):
    print("********** Test Fold " + str(i) + " ************")
    process(i)

# Test each section in process() function

In [ ]:
test_fold_number = 0
# train has 9 folds, test has 1 fold
train_data = np.concatenate((all_data[:test_fold_number*fold_n], all_data[fold_n+test_fold_number*fold_n:]), axis=0)
train_labels = np.concatenate((all_labels[:test_fold_number*fold_n], all_labels[fold_n+test_fold_number*fold_n:]), axis=0)
test_data = all_data[test_fold_number*fold_n : fold_n+test_fold_number*fold_n]
test_labels = all_labels[test_fold_number*fold_n : fold_n+test_fold_number*fold_n]
print(train_data.shape,test_data.shape) # (945, 32, 8064) (105, 32, 8064)

In [ ]:
# -------- Feature extraction from 32 original signal --------
train_band_power = [] # band power feature sequence for train trials
for data in train_data: # for every train trial
    with io.capture_output() as captured:
        trial_band_power = trial_psd_extraction_integration(data) # data shape (32, 8064)
    train_band_power.append(trial_band_power)
train_band_power = np.array(train_band_power)

test_band_power = [] # band power feature sequence for test trials
for data in test_data: # for every test trial
    with io.capture_output() as captured:
        trial_band_power = trial_psd_extraction_integration(data) # data shape (32, 8064)
    test_band_power.append(trial_band_power)
test_band_power = np.array(test_band_power)
print("All features of training data shape: ", train_band_power.shape) # shape (849, 125, 128)
print("All features of test data shape: ", test_band_power.shape) # shape (95, 125, 128)

In [ ]:
# -------- Create new LSTM model --------
x=Input(shape=(n_segment,bottleneck*4)) # flatten (12,4) to 48
x1=LSTM(n_segment)(x)
x2=Dense(n_segment)(x1)
x3=Dense(12)(x2)
output=Dense(1, activation="sigmoid")(x2)
model=Model(x, output)

# -------- Compile and train LSTM --------
model.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])
model.fit(train_band_power, train_labels, epochs=30, batch_size=8, validation_data=(test_band_power, test_labels))
model.save("../Results/LSTM_model/LSTM_model_test_fold_" + str(test_fold_number))